# Core Model Guide: XinanjiangModel

This notebook provides a guide to the `XinanjiangModel`, a classic and influential conceptual rainfall-runoff model. It is particularly well-suited for humid and semi-humid regions.

## 1. Theoretical Background

The Xinanjiang model is based on the concept of **saturation excess runoff**. This means that runoff is generated when the soil becomes fully saturated and can no longer absorb more water. This is different from the infiltration excess mechanism (where runoff occurs when rainfall intensity exceeds the soil's infiltration capacity).

Key concepts of the model include:

- **Tension Water Capacity**: The model divides the soil into upper, lower, and deep layers, each with its own tension water capacity. The distribution of this capacity over the catchment area is assumed to be non-uniform, which is a core idea of the model.
- **Three Soil Layers**: The model considers evaporation from three soil layers: upper, lower, and deep.
- **Runoff Components**: The total runoff is separated into different components, such as surface runoff, interflow, and groundwater flow, which are then routed to the catchment outlet.

## 2. API and Parameters

The `XinanjiangModel` in the CHS SDK implements the core runoff generation part of the model. Its `calculate_runoff` method uses the following key parameters, typically provided in a `sub_basin_params` dictionary:

- `WM`: The average tension water capacity of the catchment (mm).
- `B`: The exponent of the tension water capacity curve (dimensionless).
- `IM`: The fraction of the catchment area that is impervious (dimensionless).

The model also maintains an internal state:

- `W`: The current tension water content of the soil (mm).

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.modeling.hydrology.runoff_models import XinanjiangModel

## 3. Code Example: Single Step Calculation

Due to the model's complexity, we will demonstrate a single time step calculation to show how rainfall is converted to runoff and how the soil moisture state is updated. We will manually verify the calculation.

In [ ]:
# 1. Model and Parameter Setup
params = {"WM": 100.0, "B": 0.3}
initial_W = 50.0  # Assume soil is 50% saturated
rainfall = 20.0   # 20mm of rain in this time step

model = XinanjiangModel(states={"initial_W": initial_W})

# 2. Manual Calculation (for verification)
W = initial_W
WM = params["WM"]
B = params["B"]

# Potential total tension water capacity
WMM = WM * (1 + B)
# Tension water capacity of the point where soil is saturated
A = WMM * (1 - (1 - W / WM)**(1 / (1 + B)))

# Check if rainfall exceeds storage capacity
if rainfall + A >= WMM:
    expected_runoff = rainfall - (WM - W)
else:
    term = 1 - (rainfall + A) / WMM
    expected_runoff = rainfall + W - WM + WM * (term**(1 + B))

expected_runoff = max(0, expected_runoff)
expected_final_W = W + rainfall - expected_runoff

# 3. Run the model's calculation
actual_runoff = model.calculate_runoff(rainfall, params, dt=1.0)
actual_final_W = model.W

# 4. Print and Verify
print(f"Initial Soil Moisture (W): {initial_W:.2f} mm")
print(f"Rainfall (P): {rainfall:.2f} mm")
print(f"--- Calculations ---")
print(f"Expected Runoff: {expected_runoff:.4f} mm")
print(f"Actual Runoff:   {actual_runoff:.4f} mm")
print(f"Expected Final W: {expected_final_W:.4f} mm")
print(f"Actual Final W:   {actual_final_W:.4f} mm")

# Assert that our manual calculation matches the model's output
np.testing.assert_almost_equal(expected_runoff, actual_runoff, decimal=5)
np.testing.assert_almost_equal(expected_final_W, actual_final_W, decimal=5)

The code above shows that the model's internal calculation matches the step-by-step formula for the Xinanjiang runoff generation process. By running this calculation in a loop with a time series of rainfall and evaporation data, a complete runoff hydrograph can be generated.